In [1]:
import os
os.environ['HF_HOME'] = os.path.join(os.getcwd(), 'hf_cache')
from transformers import XLNetConfig, XLNetModel, XLNetTokenizer, XLNetLMHeadModel, XLNetForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import utils
from datasets import load_metric
from transformers.data.data_collator import DataCollatorWithPadding
import numpy as np
from tqdm import tqdm
import sklearn

In [2]:
tokenizer = XLNetTokenizer(vocab_file='models/smiles_sp.model',
                           do_lower_case=False,
                           keep_accents=True
                           )

In [3]:
#NOTE: Datasets have had the last elements removed to make them even. TODO: Figure out why huggingface can't handle odd numbers. 
train_raw = load_dataset('csv', data_files=['data/ogb_molhiv/train_hiv.csv'])
test_raw = load_dataset('csv', data_files=['data/ogb_molhiv/test_hiv.csv'])
valid_raw = load_dataset('csv', data_files=['data/ogb_molhiv/valid_hiv.csv'])

Using custom data configuration default-0250d14726bc71f8


100%|██████████| 1/1 [00:00<00:00, 499.86it/s]


Dataset csv downloaded and prepared to e:\molnlp\mol-prop\hf_cache\datasets\csv\default-0250d14726bc71f8\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 100.02it/s]
Using custom data configuration default-d85d5e570e46467b


100%|██████████| 1/1 [00:00<00:00, 998.41it/s]


Dataset csv downloaded and prepared to e:\molnlp\mol-prop\hf_cache\datasets\csv\default-d85d5e570e46467b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 111.13it/s]
Using custom data configuration default-e9711672e6359f2b


100%|██████████| 1/1 [00:00<00:00, 498.49it/s]


Dataset csv downloaded and prepared to e:\molnlp\mol-prop\hf_cache\datasets\csv\default-e9711672e6359f2b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 99.99it/s]


In [4]:

def tokenize_function_hiv(examples):
    out_dict = tokenizer(examples["smiles"])
    out_dict['label'] = [int(x) for x in examples['HIV_active']]
    return out_dict

train_ds = train_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']
test_ds = test_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']
valid_ds = valid_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']

100%|██████████| 5/5 [00:00<00:00,  7.79ba/s]


In [5]:

model_path = "models/xlnet-smiles-s2000000-b70-l4/checkpoint-5714"
model = XLNetForSequenceClassification.from_pretrained(model_path, config=model_path, num_labels=2)
"""
model_config = XLNetConfig(
    vocab_size=tokenizer.vocab_size,
    n_layer=6,
    bi_data=True,
    num_labels=2
)
model = XLNetForSequenceClassification(model_config)
"""

Some weights of the model checkpoint at models/xlnet-smiles-s2000000-b70-l4/checkpoint-5714 were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at models/xlnet-smiles-s2000000-b70-l4/checkpoint-5714 and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN t

'\nmodel_config = XLNetConfig(\n    vocab_size=tokenizer.vocab_size,\n    n_layer=6,\n    bi_data=True,\n    num_labels=2\n)\nmodel = XLNetForSequenceClassification(model_config)\n'

In [7]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis=-1)
    metrics = metric.compute(predictions=predictions, references=labels)
    

    labels = np.asarray(labels)
    labels_oh = np.zeros((len(labels), max(labels)+1))
    labels_oh[np.arange(len(labels)),labels] = 1
    metrics['auc_roc'] = sklearn.metrics.roc_auc_score(labels_oh, logits)

    return metrics

In [6]:
training_args = TrainingArguments(
    f"models/xlnet-hiv",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8, # has to be even?
    per_device_eval_batch_size=30,
    num_train_epochs=10,
    save_strategy='epoch',
)

In [8]:
collator = DataCollatorWithPadding(tokenizer)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics,
    data_collator=collator,
)

In [10]:
print(trainer.evaluate())

***** Running Evaluation *****
  Num examples = 4112
  Batch size = 30
100%|██████████| 138/138 [00:28<00:00,  4.79it/s]

{'eval_loss': 0.8127949833869934, 'eval_accuracy': 0.1831225680933852, 'eval_auc_roc': 0.31710678823938493, 'eval_runtime': 29.5105, 'eval_samples_per_second': 139.34, 'eval_steps_per_second': 4.676}


In [11]:
trainer.train()

***** Running training *****
  Num examples = 32900
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 41130
  1%|          | 501/41130 [00:59<1:24:03,  8.06it/s]

{'loss': 0.1902, 'learning_rate': 1.975686846584002e-05, 'epoch': 0.12}


  2%|▏         | 1001/41130 [01:59<1:17:29,  8.63it/s]

{'loss': 0.1822, 'learning_rate': 1.951373693168004e-05, 'epoch': 0.24}


  4%|▎         | 1501/41130 [02:59<1:23:23,  7.92it/s]

...

 53%|█████▎    | 22001/41130 [44:35<36:23,  8.76it/s]

{'loss': 0.1341, 'learning_rate': 9.302212496960856e-06, 'epoch': 5.35}


 55%|█████▍    | 22501/41130 [45:33<46:28,  6.68it/s]

{'loss': 0.1451, 'learning_rate': 9.059080962800875e-06, 'epoch': 5.47}


 55%|█████▍    | 22517/41130 [45:35<35:29,  8.74it/s]